In [16]:
import pickle
import en_core_web_sm
import spacy

In [17]:
filename = '../training_data'
infile = open(filename,'rb')
TRAIN_DATA = pickle.load(infile)

In [18]:
nlp=spacy.load('en_core_web_sm')

In [25]:
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner=nlp.create_pipe('ner')
        nlp.add_pipe(ner,last=True)
    else:
        ner = nlp.get_pipe("ner")
    for _,annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
    


    other_pipes = [pipe for pipe in nlp.pipe_names if pipe !='ner']
    # only train NER
    with nlp.disable_pipes(*other_pipes):
        optimizer=nlp.begin_training()
        # show warnings for misaligned entity spans once
        # warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        # if model is None:
        #     nlp.begin_training()
        for itn in range(10):
            random.shuffle(TRAIN_DATA)
            losses = {}
            index=0
            for text,annotations in TRAIN_DATA:
                try:
                    nlp.update(
                        [texts],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,
                        sgd=optimizer,  # dropout - make it harder to memorise data
                        losses=losses)
                except Exception as e:
                    pass
            print(losses)
            # batch up the examples using spaCy's minibatch
            # batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            # for batch in batches:
            #     texts, annotations = zip(*batch)
            #     nlp.update(
            #         texts,  # batch of texts
            #         annotations,  # batch of annotations
            #         drop=0.5,  # dropout - make it harder to memorise data
            #         losses=losses,
            #     )
            # print("Losses", losses)
    

In [26]:
train_model(TRAIN_DATA)

NameError: name 'n_iter' is not defined